In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from os.path import join as p_join

import seaborn as sns


##################################
## GLOBAL SETTINGS ###############
##################################
plt.rcParams["figure.figsize"] = (12,8)

In [2]:
### Install CPU torch
# ! pip3 install torch==1.10.2+cpu torchvision==0.11.3+cpu torchaudio==0.10.2+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html
    
### Inctall GPU torch (CUDA 11.3)
# ! pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [3]:
###############################
##### IMPORT DL Depensies #####
###############################
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader

from tqdm import tqdm


def seed_all(seed=42):

    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print("[ Using Seed : ", seed, " ]")

####################################
#####   SEED ALL EXPERIMENTS   #####
####################################
seed_all()    


#################################
####### GLOBAL CONFIG ###########
#################################
CONFIG = {'ephs': 100,
          'train_batch_size': 256,
          'test_batch_size': 128,
          'cpu_workers': 2,
          'save_ckpts': True,
          'ckpt_save_folder': 'cnn_ckpts_L_8_6'}

[ Using Seed :  42  ]


In [4]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utils import utils

In [5]:
data_path = p_join('..', '..', 'data_2')
FOLDERS = [p_join(os.path.abspath(data_path), item) for item in os.listdir(data_path) if 'L=8.6' in item]
FOLDERS

['/home/akhorkin/Repositories/ml_dl_with_topology_data/data_2/N2=27_L=8.6_input=2',
 '/home/akhorkin/Repositories/ml_dl_with_topology_data/data_2/N2=25_L=8.6_input=-4',
 '/home/akhorkin/Repositories/ml_dl_with_topology_data/data_2/N2=25_L=8.6_input=-2',
 '/home/akhorkin/Repositories/ml_dl_with_topology_data/data_2/N2=25_L=8.6_input=4',
 '/home/akhorkin/Repositories/ml_dl_with_topology_data/data_2/N2=27_L=8.6_input=-4',
 '/home/akhorkin/Repositories/ml_dl_with_topology_data/data_2/N2=23_L=8.6_input=4',
 '/home/akhorkin/Repositories/ml_dl_with_topology_data/data_2/N2=23_L=8.6_input=-2',
 '/home/akhorkin/Repositories/ml_dl_with_topology_data/data_2/N2=21_L=8.6_input=-4',
 '/home/akhorkin/Repositories/ml_dl_with_topology_data/data_2/N2=27_L=8.6_input=-2',
 '/home/akhorkin/Repositories/ml_dl_with_topology_data/data_2/N2=23_L=8.6_input=-4',
 '/home/akhorkin/Repositories/ml_dl_with_topology_data/data_2/N2=21_L=8.6_input=2',
 '/home/akhorkin/Repositories/ml_dl_with_topology_data/data_2/N2=25_L

### Create dataset

In [6]:
X, Y = utils.create_dataset(FOLDERS)
X = utils.normalize_data(X)
X = np.expand_dims(X, axis=1)

#################################
#### MAKE TEST TRAIN SPLIT   ####
#################################
def create_dataloaders(CONFIG, X, Y):
    from sklearn.model_selection import train_test_split

    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.2, random_state=42)


    train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(Y_train))
    test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(Y_test))

    train_dataloader = DataLoader(train_dataset,
                                  shuffle=True,
                                  num_workers=CONFIG['cpu_workers'],
                                  batch_size=CONFIG['train_batch_size'],
                                  drop_last=True)

    test_dataloader = DataLoader(test_dataset,
                                 shuffle=False,
                                 num_workers=CONFIG['cpu_workers'],
                                 batch_size=CONFIG['test_batch_size'],
                                 drop_last=False)

    return train_dataloader, test_dataloader

100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 22.51it/s]


### Try more layers

In [7]:
class Block(nn.Module):
    def __init__(self, inp, out, ks):
        super().__init__()
        self.activ = F.gelu
        self.conv1 = nn.Conv1d(inp, out, ks, padding=1)
        self.bn1 = nn.BatchNorm1d(out)

    def forward(self, x):
        x = self.activ(self.bn1(self.conv1(x)))
        return x

In [8]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.drop_p = 0.1
        self.h_dim = 240
        self.h_dims = [1, 32, 64, self.h_dim]
        self.backbone = nn.Sequential(*[Block(self.h_dims[i], self.h_dims[i + 1], 3) for i in range(len(self.h_dims) - 1)])
        self.flat = nn.Flatten()
        self.drop = nn.Dropout(p=self.drop_p)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.lin = nn.Sequential(nn.Linear(self.h_dim, self.h_dim * 2),
                                 nn.BatchNorm1d(self.h_dim * 2),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(self.h_dim * 2, self.h_dim // 2),
                                 nn.BatchNorm1d(self.h_dim // 2),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(self.h_dim // 2, 4))

    def forward(self, x):
        x = self.backbone(x)
        x = self.pool(x)
        x = self.flat(x)
        x = self.drop(x)
#         print(x.shape)
        x = self.lin(x)
        return x
    
model = ConvNet()
inp = torch.rand(32, 1, 16)
out = model(inp)
out.shape


net = ConvNet()
train_dataloader, test_dataloader = create_dataloaders(CONFIG, X, Y)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 3e-3))
utils.run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

100%|█████████████████████████████████████████| 199/199 [00:10<00:00, 18.21it/s]


Epoch: 1/100, 	 total train loss: 1.1642510157134665


100%|████████████████████████████████████████| 100/100 [00:00<00:00, 113.81it/s]


Epoch: 1/100, 	 total score test: 45.00273501601938, [best score: 45.00273501601938]



100%|█████████████████████████████████████████| 199/199 [00:10<00:00, 18.23it/s]


...

Epoch: 99/100, 	 total score test: 72.70454012659216, [best score: 73.80636086582793]



100%|█████████████████████████████████████████| 199/199 [00:14<00:00, 14.18it/s]


Epoch: 100/100, 	 total train loss: 0.28520079631002704


100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 82.03it/s]

Epoch: 100/100, 	 total score test: 72.79049777291553, [best score: 73.80636086582793]

